# Import libs

In [1]:
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from VisionDatasets import ContactDataset
from tqdm import tqdm
from PIL import Image
from datetime import datetime

import utils
from VisionModels import CustomCNN
%load_ext autoreload
%autoreload 2

# Pre-settings

In [2]:
parameters_json = os.path.join(os.getcwd(), 'settings/parameters.json')
paths_json = os.path.join(os.getcwd(), 'settings/paths.json')

In [3]:
with open(parameters_json, 'r') as json_file:
    parameters = json.load(json_file)

with open(paths_json, 'r') as json_file:
    paths = json.load(json_file)

In [4]:
random_seed = parameters["random_seed"]

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

In [5]:
train_set = [1, 3, 5, 7, 8, 10, 12, 14, 15, 17, 19, 21, 41, 98, 107, 143]
val_set = [2, 4, 9, 11, 16, 18, 42, 116]
test_set = [37, 68, 71, 23, 22, 24, 25, 26, 27, 28, 29,
            46, 47, 38, 95, 104, 101, 92, 110, 45, 125, 134]

# Import Data

In [6]:
data = pd.read_csv(paths['silicone']['labels'], header=0)

In [7]:
train_data = data[data['HPC_Path'].str.extract(
    r'imageset_(\d+)').astype(int).isin(train_set).any(axis=1)]
val_data = data[data['HPC_Path'].str.extract(
    r'imageset_(\d+)').astype(int).isin(val_set).any(axis=1)]
test_data = data[data['HPC_Path'].str.extract(
    r'imageset_(\d+)').astype(int).isin(test_set).any(axis=1)]

# Training

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CustomCNN(random_seed=random_seed)

optimizer = optim.Adam(
    model.parameters(),
    lr=parameters['CustomCNN']['learning_rate'],
    weight_decay=parameters['CustomCNN']['weight_decay'])

loss_fn = nn.CrossEntropyLoss()

train_dataloader = DataLoader(
    dataset=ContactDataset(
        images=[Image.open(image)
                for image in train_data['HPC_Path'].tolist()],
        labels=train_data['GT'].to_numpy()
    ),
    batch_size=parameters['CustomCNN']['batch_size'],
    pin_memory=True,
    num_workers=(4 if os.cpu_count() > 4 else os.cpu_count()))

val_dataloader = DataLoader(
    dataset=ContactDataset(
        images=[Image.open(image)
                for image in val_data['HPC_Path'].tolist()],
        labels=val_data['GT'].to_numpy()
    ),
    batch_size=1024,
    pin_memory=True)

In [9]:
metrics = utils.train(model=model,
            optimizer=optimizer,
            loss_fn=loss_fn,
            dataloader=train_dataloader,
            val_dataloader=val_dataloader,
            device=device,
            use_tqdm=True,
            epochs=100)

Epoch 99: 100%|██████████| 3400/3400 [01:25<00:00, 39.69it/s, loss=0.182]


In [11]:
# Get the current time and format it as a string
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")

with open(f'results/metrics_{timestamp}.json', 'w') as f:
    json.dump(metrics, f)